# Reconhecimento de Armas Brancas
Nesse projeto temos como objetivo a identificação de armas brancas em determinadas imagens ou mesmo videos.
Este é o projeto do Hackathon postech fiap.

**Integrantes**
Rodrigo Roque Vidal :
rodrigo.vidal@fbsbrasil.com.br
/
Gabriel Campioto
gcampioto@gmail.com


Conectando google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preparando dataset para treinamento
Extraindo Arquivo zip com dataset de imagens
Neste primeiro dataset temos amostras apenas de facas e canivetes
Vamos treinar nosso modelo com ele e depois treinamos com armas de fogo
Abaixo link para download do dataset utilizado.
https://universe.roboflow.com/morcik-fix/weapon-detection-knifes-mohip-kmkw6

mais informações sobre o dataset podem ser encontradas em:
./datasets/README.dataset.txt
./datasets/README.roboflow.txt

In [ ]:
import zipfile
import shutil
import os

# Caminho original do ZIP no Google Drive
zip_path = "/content/drive/MyDrive/postech/challenge4/datasets/weapon-detection-knifes.yolov5pytorch.zip"

# Caminho para armazenamento local no Colab
local_zip = "/content/weapon-detection.zip"
local_extract_path = "/content/weapon-detection/"

# Passo 1: Copiar o ZIP do Drive para o Colab (rápido)
shutil.copy(zip_path, local_zip)

# Passo 2: Extrair no armazenamento local do Colab
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall(local_extract_path)

print(f"Arquivos extraídos para: {local_extract_path}")


Arquivos extraídos para: /content/weapon-detection/


In [ ]:
# Caminhos corrigidos para a extração local
data_yaml = f"""
train: /content/weapon-detection/train/images
val: /content/weapon-detection/valid/images

nc: 1  # Número de classes
names: ['arma_branca']  # Nomes das classes
"""

# Salvar o arquivo data.yaml no local correto
data_yaml_path = "/content/weapon-detection/data.yaml"

with open(data_yaml_path, "w") as file:
    file.write(data_yaml)

print(f"Arquivo data.yaml salvo em: {data_yaml_path}")


Arquivo data.yaml salvo em: /content/weapon-detection/data.yaml


# Treinar o Modelo com o dataset baixado ( Armas Brancas )



In [ ]:
# Instalar o pacote ultralytics para usar YOLO
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
from ultralytics import YOLO

# 1️⃣ Carregar o modelo YOLO pré-treinado
os.environ['YOLO_RESULTS_DIR'] = "/content/runs"
model = YOLO("yolov5s.pt")  # Se tiver um modelo já treinado, substitua pelo caminho dele

# 2️⃣ Caminho atualizado do data.yaml
data_yaml_path = "/content/weapon-detection/data.yaml"

# 3️⃣ Iniciar o treinamento com os dados locais (evita lentidão do Drive)
model.train(
    data="/content/weapon-detection/data.yaml",
    epochs=100,  # Teste aumentar para 100
    imgsz=640,
    batch=16,
    patience=10,  # Para early stopping automático
    lr0=0.001,  # Taxa de aprendizado inicial
    optimizer="SGD"  # Testar SGD ou Adam
)

# 4️⃣ Salvar o modelo treinado no ambiente do Colab (mais rápido)
model_path = "/content/armas_brancas.pt"
model.save(model_path)

# 5️⃣ Mover para o Google Drive (somente após o treinamento)
import shutil
shutil.move(model_path, "/content/drive/MyDrive/postech/challenge4/armas_brancas.pt")

print("Modelo treinado salvo no Google Drive com sucesso!")


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/content/weapon-detection/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots

100%|██████████| 755k/755k [00:00<00:00, 99.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 354MB/s]


AMP: checks passed ✅


...

     48/100      4.18G     0.9445     0.6483      1.127         21        640: 100%|██████████| 1111/1111 [01:52<00:00,  9.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:07<00:00,  8.26it/s]

                   all       1960       2014      0.743      0.755      0.663      0.387


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 38, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

48 epochs completed in 1.612 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 18.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 18.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv5s summary (fused): 193 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:08<00:00,  7.25it/s]


                   all       1960       2014      0.741      0.763      0.676      0.392
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train
Modelo treinado salvo no Google Drive com sucesso!


In [ ]:
import shutil

# Caminho onde o YOLO salva os modelos
yolo_model_path = "/content/runs/detect/train/weights/best.pt"

# Nome final do modelo salvo no Google Drive
final_model_path = "/content/drive/MyDrive/postech/challenge4/armas_brancas_best.pt"

# nome final do modelo salvo no local
final_model_local = "/content/armas_brancas_best.pt"


# Mover o melhor modelo para o local
shutil.copy(yolo_model_path, final_model_local)

# Mover o melhor modelo para o Google Drive
shutil.copy(yolo_model_path, final_model_path)

print(f"Modelo salvo no Google Drive como {final_model_path}")


Modelo salvo no Google Drive como /content/drive/MyDrive/postech/challenge4/armas_brancas_best.pt


# Testando modelo treinado

In [ ]:
from ultralytics import YOLO

# Carregar o modelo treinado
model = YOLO("/content/armas_brancas_best.pt")  # Altere para o caminho do seu modelo

# Diretório das imagens de teste
test_images_path = "/content/weapon-detection/test/images/"

# Fazer inferência em todas as imagens
results = model.predict(source=test_images_path, save=True, conf=0.5)

print("Inferência concluída! Imagens com detecções salvas.")



image 1/502 /content/weapon-detection/test/images/20221010_222416_jpg.rf.aa4f1793845bc740d68e6d90fa777d61.jpg: 640x320 1 arma_branca, 54.8ms
image 2/502 /content/weapon-detection/test/images/20221010_222456_jpg.rf.25a7374fffcaa0790ee213394cbac9b7.jpg: 640x320 1 arma_branca, 8.5ms
image 3/502 /content/weapon-detection/test/images/20221013_145744_jpg.rf.44f2efc6fbd5458cd020735765082178.jpg: 640x288 1 arma_branca, 54.6ms
image 4/502 /content/weapon-detection/test/images/20221013_145748-0-_jpg.rf.e27745b289c60aaab9ddbcb4332d1691.jpg: 640x288 1 arma_branca, 8.3ms
image 5/502 /content/weapon-detection/test/images/20221013_145802_jpg.rf.fd27ac0abc79f8072b61b3a053681b79.jpg: 640x288 1 arma_branca, 8.7ms
image 6/502 /content/weapon-detection/test/images/20221013_145829-0-_jpg.rf.74a26dec650679b789208cc4547f53fe.jpg: 640x288 1 arma_branca, 8.4ms
image 7/502 /content/weapon-detection/test/images/20221013_145834_jpg.rf.7139b202eadb333e74ac59bdd5a2a56b.jpg: 640x288 1 arma_branca, 8.4ms
image 8/502

In [ ]:
# Avaliar o desempenho do modelo
metrics = model.val(data="/content/weapon-detection/data.yaml")

# Exibir métricas importantes
print(f"Precisão (Precision): {metrics.box.map50:.2f}")
print(f"Recall: {metrics.box.map75:.2f}")
print(f"mAP@50: {metrics.box.map50:.2f}")
print(f"mAP@50-95: {metrics.box.map:.2f}")


Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)


val: Scanning /content/weapon-detection/valid/labels.cache... 1960 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1960/1960 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 2014. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 123/123 [00:10<00:00, 12.07it/s]


                   all       1960       2014       0.74       0.76      0.674      0.393
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val
Precisão (Precision): 0.67
Recall: 0.41
mAP@50: 0.67
mAP@50-95: 0.39


As imagens com bounding boxes serão salvas na pasta /content/runs/detect/predict/

As estatísticas detalhadas serão salvas no relatório gerado pelo model.val()

In [ ]:
import shutil

# Mover imagens processadas para o Drive
shutil.copytree("/content/runs/detect/predict/", "/content/drive/MyDrive/postech/challenge4/resultados2/")

# Mover estatísticas para o Drive
shutil.copytree("/content/runs/detect/val/", "/content/drive/MyDrive/postech/challenge4/metricas2/")

print("Resultados salvos no Google Drive!")


Resultados salvos no Google Drive!
